In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import StratifiedKFold



import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

c:\Users\badr0\.conda\envs\py310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\badr0\.conda\envs\py310\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want

In [2]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Parameters
img_width, img_height = 224, 224
batch_size = 32
epochs = 50

# Directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Data generator for training data

# Data generator for training data with validation split
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # 20% validation split

# Separate generator for training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # Specify 'training' subset

# Separate generator for validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # Specify 'validation' subset



# Data generator for test data
test_files = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]

# Create a DataFrame with the filenames
test_df = pd.DataFrame({"Filename": test_files})

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='Filename',
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)


Found 7920 images belonging to 5 classes.
Found 1980 images belonging to 5 classes.
Found 100 validated image filenames.


In [3]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Normalization(),
        layers.experimental.preprocessing.Resizing(224, 224),
#         layers.experimental.preprocessing.RandomFlip("horizontal"),
#         layers.experimental.preprocessing.RandomRotation(factor=0.02),
#         layers.experimental.preprocessing.RandomZoom(
#             height_factor=0.2, width_factor=0.2
#         ),
    ],
    name="data_augmentation",
)

In [4]:
from vit_keras import vit, utils

def create_vit_classifier():
    inputs = layers.Input(shape=(224,224,3))
    # Augment data.
    augmented = data_augmentation(inputs)


    pre_trained_model = vit.vit_l32(
        image_size=224,
        activation='relu',
        pretrained=True,
        include_top=True,
        pretrained_top=True,
        classes=5
    )
    pre_trained_model.trainable = False
    features = pre_trained_model(augmented)
    features = layers.Dense(512, activation=layers.PReLU())(features)
    features = layers.Dropout(0.2)(features)
    features = layers.Dense(256, activation=layers.PReLU())(features)
    features = layers.Dropout(0.2)(features)
    features = layers.Dense(128, activation=layers.PReLU())(features)
    features = layers.Dropout(0.2)(features)

    outputs = layers.Dense(5, activation="softmax")(features)
    model = keras.Model(inputs=inputs,outputs=outputs)
    return model

In [ ]:
def run_experiment(model):



    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




    history = model.fit(
        train_generator,
        batch_size=batch_size,
        epochs=10,
        validation_data=validation_generator,

    )
    

    model.trainable = True
    history = model.fit(
        train_generator,
        batch_size=batch_size,
        epochs=10 // 2,

        validation_data=validation_generator,

    )

    accuracy = model.evaluate(test_generator)
    


    #model.save("./vit_best_model.h5")
    return history

vit_classifier = create_vit_classifier()
vit_classifier.summary()
history = run_experiment(vit_classifier)


_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      7         
 al)                                                             
                                                                 
 vit-l32 (Functional)        (None, 1000)              306535400 
                                                                 
 dense (Dense)               (None, 512)               513024    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               33024     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 5)                 645       
                                                                 
=================================================================
Total params: 307,213,684
Trainable params: 678,277
Non-trainable params: 306,535,407
_________________________________________________________________
Epoch 1/10
248/248 [==============================] - 112s 353ms/step - loss: 0.8325 - accuracy: 0.6831 - val_loss: 0.6446 - val_accuracy: 0.7798
Epoch 2/10
248/248 [==============================] - 81s 328ms/step - loss: 0.5841 - accuracy: 0.7890 - val_loss: 0.5832 - val_accuracy: 0.7955
Epoch 3/10
248/248 [==============================] - 85s 342ms/step - loss: 0.5336 - accuracy: 0.8112 - val_loss: 0.5466 - val_accuracy: 0.8035
Epoch 4/10
248/248 [==============================] - 80s 321ms/step - loss: 0.4998 - accuracy: 0.8212 - val_loss: 0.5276 - val_accuracy: 0.8187
Epoch 5/10
248/248 [==============================] - 83s 336ms/step - loss: 0.4567 - accuracy: 0.8302 - val_loss: 0.4964 - val_accuracy: 0.8217
Epoch 6/10
248/248 [==============================] - 81s 326ms/step - loss: 0.4556 - accuracy: 0.8388 - val_loss: 0.5056 - val_accuracy: 0.8157
Epoch 7/10
248/248 [==============================] - 77s 309ms/step - loss: 0.4236 - accuracy: 0.8489 - val_loss: 0.4799 - val_accuracy: 0.8308
Epoch 8/10
248/248 [==============================] - 79s 317ms/step - loss: 0.4301 - accuracy: 0.8456 - val_loss: 0.4690 - val_accuracy: 0.8323
Epoch 9/10
248/248 [==============================] - 85s 341ms/step - loss: 0.4060 - accuracy: 0.8564 - val_loss: 0.4698 - val_accuracy: 0.8414
Epoch 10/10
248/248 [==============================] - 110s 443ms/step - loss: 0.4046 - accuracy: 0.8537 - val_loss: 0.4880 - val_accuracy: 0.8242
Epoch 1/5
248/248 [==============================] - 88s 350ms/step - loss: 0.3933 - accuracy: 0.8626 - val_loss: 0.4923 - val_accuracy: 0.8278
Epoch 2/5
248/248 [==============================] - 88s 349ms/step - loss: 0.3870 - accuracy: 0.8629 - val_loss: 0.4457 - val_accuracy: 0.8515
Epoch 3/5
248/248 [==============================] - 81s 320ms/step - loss: 0.3834 - accuracy: 0.8684 - val_loss: 0.4708 - val_accuracy: 0.8419
Epoch 4/5
248/248 [==============================] - 81s 324ms/step - loss: 0.3758 - accuracy: 0.8638 - val_loss: 0.4453 - val_accuracy: 0.8404
Epoch 5/5
248/248 [==============================] - 81s 324ms/step - loss: 0.3660 - accuracy: 0.8678 - val_loss: 0.4467 - val_accuracy: 0.8460
4/4 [==============================] - 5s 41ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


TRANSFORMER
Epoch 1/40
248/248 [==============================] - 104s 338ms/step - loss: 0.8277 - accuracy: 0.6889 - val_loss: 0.5984 - val_accuracy: 0.7874
Epoch 2/40
248/248 [==============================] - 77s 308ms/step - loss: 0.5821 - accuracy: 0.7907 - val_loss: 0.5760 - val_accuracy: 0.7960
Epoch 3/40
248/248 [==============================] - 77s 310ms/step - loss: 0.5331 - accuracy: 0.8080 - val_loss: 0.5695 - val_accuracy: 0.7869
Epoch 4/40
248/248 [==============================] - 77s 309ms/step - loss: 0.4952 - accuracy: 0.8231 - val_loss: 0.5507 - val_accuracy: 0.8066
Epoch 5/40
248/248 [==============================] - 77s 309ms/step - loss: 0.4642 - accuracy: 0.8309 - val_loss: 0.4866 - val_accuracy: 0.8293
Epoch 6/40
248/248 [==============================] - 76s 308ms/step - loss: 0.4453 - accuracy: 0.8402 - val_loss: 0.5154 - val_accuracy: 0.8152
Epoch 7/40
248/248 [==============================] - 77s 308ms/step - loss: 0.4361 - accuracy: 0.8441 - val_loss: 0.5189 - val_accuracy: 0.8192
Epoch 8/40
248/248 [==============================] - 77s 311ms/step - loss: 0.4208 - accuracy: 0.8494 - val_loss: 0.4994 - val_accuracy: 0.8187
Epoch 9/40
248/248 [==============================] - 77s 310ms/step - loss: 0.4105 - accuracy: 0.8556 - val_loss: 0.5434 - val_accuracy: 0.8172
Epoch 10/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3963 - accuracy: 0.8591 - val_loss: 0.4925 - val_accuracy: 0.8222
Epoch 11/40
248/248 [==============================] - 77s 310ms/step - loss: 0.3941 - accuracy: 0.8629 - val_loss: 0.4630 - val_accuracy: 0.8288
Epoch 12/40
248/248 [==============================] - 76s 308ms/step - loss: 0.3822 - accuracy: 0.8662 - val_loss: 0.4490 - val_accuracy: 0.8419
Epoch 13/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3866 - accuracy: 0.8586 - val_loss: 0.4578 - val_accuracy: 0.8404
Epoch 14/40
248/248 [==============================] - 76s 305ms/step - loss: 0.3788 - accuracy: 0.8655 - val_loss: 0.4499 - val_accuracy: 0.8480
Epoch 15/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3578 - accuracy: 0.8766 - val_loss: 0.4207 - val_accuracy: 0.8444
Epoch 16/40
248/248 [==============================] - 76s 306ms/step - loss: 0.3483 - accuracy: 0.8749 - val_loss: 0.5112 - val_accuracy: 0.8298
Epoch 17/40
248/248 [==============================] - 77s 310ms/step - loss: 0.3572 - accuracy: 0.8744 - val_loss: 0.4907 - val_accuracy: 0.8313
Epoch 18/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3533 - accuracy: 0.8777 - val_loss: 0.4622 - val_accuracy: 0.8348
Epoch 19/40
248/248 [==============================] - 78s 312ms/step - loss: 0.3546 - accuracy: 0.8759 - val_loss: 0.4387 - val_accuracy: 0.8475
Epoch 20/40
248/248 [==============================] - 80s 323ms/step - loss: 0.3412 - accuracy: 0.8756 - val_loss: 0.4191 - val_accuracy: 0.8455
Epoch 21/40
248/248 [==============================] - 78s 315ms/step - loss: 0.3386 - accuracy: 0.8793 - val_loss: 0.4500 - val_accuracy: 0.8429
Epoch 22/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3221 - accuracy: 0.8893 - val_loss: 0.4427 - val_accuracy: 0.8490
Epoch 23/40
248/248 [==============================] - 76s 306ms/step - loss: 0.3431 - accuracy: 0.8814 - val_loss: 0.4141 - val_accuracy: 0.8566
Epoch 24/40
248/248 [==============================] - 81s 325ms/step - loss: 0.3269 - accuracy: 0.8857 - val_loss: 0.4758 - val_accuracy: 0.8505
Epoch 25/40
248/248 [==============================] - 80s 323ms/step - loss: 0.3320 - accuracy: 0.8830 - val_loss: 0.4240 - val_accuracy: 0.8545
Epoch 26/40
248/248 [==============================] - 77s 311ms/step - loss: 0.3164 - accuracy: 0.8891 - val_loss: 0.4258 - val_accuracy: 0.8540
Epoch 27/40
248/248 [==============================] - 80s 322ms/step - loss: 0.3174 - accuracy: 0.8919 - val_loss: 0.4470 - val_accuracy: 0.8500
Epoch 28/40
248/248 [==============================] - 77s 310ms/step - loss: 0.3044 - accuracy: 0.8929 - val_loss: 0.4503 - val_accuracy: 0.8535
Epoch 29/40
248/248 [==============================] - 77s 309ms/step - loss: 0.3194 - accuracy: 0.8883 - val_loss: 0.4477 - val_accuracy: 0.8545
Epoch 30/40
248/248 [==============================] - 77s 308ms/step - loss: 0.3119 - accuracy: 0.8909 - val_loss: 0.4205 - val_accuracy: 0.8495
Epoch 31/40
248/248 [==============================] - 77s 309ms/step - loss: 0.2947 - accuracy: 0.8961 - val_loss: 0.4215 - val_accuracy: 0.8566
Epoch 32/40
248/248 [==============================] - 77s 310ms/step - loss: 0.2862 - accuracy: 0.8987 - val_loss: 0.4422 - val_accuracy: 0.8455
Epoch 33/40
248/248 [==============================] - 77s 309ms/step - loss: 0.3128 - accuracy: 0.8934 - val_loss: 0.4183 - val_accuracy: 0.8581
Epoch 34/40
248/248 [==============================] - 76s 308ms/step - loss: 0.2953 - accuracy: 0.8952 - val_loss: 0.4067 - val_accuracy: 0.8667
Epoch 35/40
248/248 [==============================] - 77s 310ms/step - loss: 0.2937 - accuracy: 0.8981 - val_loss: 0.4417 - val_accuracy: 0.8525
Epoch 36/40
248/248 [==============================] - 78s 313ms/step - loss: 0.2927 - accuracy: 0.8979 - val_loss: 0.4562 - val_accuracy: 0.8439
Epoch 37/40
248/248 [==============================] - 79s 317ms/step - loss: 0.2903 - accuracy: 0.9000 - val_loss: 0.4307 - val_accuracy: 0.8545
Epoch 38/40
248/248 [==============================] - 81s 326ms/step - loss: 0.2911 - accuracy: 0.8966 - val_loss: 0.4528 - val_accuracy: 0.8530
Epoch 39/40
248/248 [==============================] - 78s 313ms/step - loss: 0.2829 - accuracy: 0.9024 - val_loss: 0.4411 - val_accuracy: 0.8515
Epoch 40/40
248/248 [==============================] - 77s 309ms/step - loss: 0.2863 - accuracy: 0.8958 - val_loss: 0.5121 - val_accuracy: 0.8399
Epoch 1/20
248/248 [==============================] - 77s 311ms/step - loss: 0.2772 - accuracy: 0.9028 - val_loss: 0.4174 - val_accuracy: 0.8596
Epoch 2/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2745 - accuracy: 0.9078 - val_loss: 0.4680 - val_accuracy: 0.8343
Epoch 3/20
248/248 [==============================] - 77s 311ms/step - loss: 0.2734 - accuracy: 0.9042 - val_loss: 0.4889 - val_accuracy: 0.8444
Epoch 4/20
248/248 [==============================] - 77s 308ms/step - loss: 0.2788 - accuracy: 0.9057 - val_loss: 0.4051 - val_accuracy: 0.8626
Epoch 5/20
248/248 [==============================] - 77s 310ms/step - loss: 0.2772 - accuracy: 0.9056 - val_loss: 0.4664 - val_accuracy: 0.8530
Epoch 6/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2738 - accuracy: 0.9053 - val_loss: 0.5409 - val_accuracy: 0.8258
Epoch 7/20
248/248 [==============================] - 77s 311ms/step - loss: 0.2612 - accuracy: 0.9082 - val_loss: 0.4639 - val_accuracy: 0.8591
Epoch 8/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2690 - accuracy: 0.9038 - val_loss: 0.4245 - val_accuracy: 0.8662
Epoch 9/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2611 - accuracy: 0.9087 - val_loss: 0.5023 - val_accuracy: 0.8419
Epoch 10/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2614 - accuracy: 0.9116 - val_loss: 0.4963 - val_accuracy: 0.8384
Epoch 11/20
248/248 [==============================] - 77s 310ms/step - loss: 0.2718 - accuracy: 0.9043 - val_loss: 0.4471 - val_accuracy: 0.8505
Epoch 12/20
248/248 [==============================] - 77s 308ms/step - loss: 0.2613 - accuracy: 0.9095 - val_loss: 0.4336 - val_accuracy: 0.8561
Epoch 13/20
248/248 [==============================] - 77s 309ms/step - loss: 0.2543 - accuracy: 0.9111 - val_loss: 0.4350 - val_accuracy: 0.8455
Epoch 14/20
248/248 [==============================] - 77s 308ms/step - loss: 0.2552 - accuracy: 0.9086 - val_loss: 0.4328 - val_accuracy: 0.8556
Epoch 15/20
248/248 [==============================] - 80s 321ms/step - loss: 0.2379 - accuracy: 0.9165 - val_loss: 0.4548 - val_accuracy: 0.8505
Epoch 16/20
248/248 [==============================] - 79s 319ms/step - loss: 0.2585 - accuracy: 0.9109 - val_loss: 0.4483 - val_accuracy: 0.8556
Epoch 17/20
248/248 [==============================] - 77s 310ms/step - loss: 0.2436 - accuracy: 0.9162 - val_loss: 0.4635 - val_accuracy: 0.8561
Epoch 18/20
248/248 [==============================] - 77s 308ms/step - loss: 0.2471 - accuracy: 0.9155 - val_loss: 0.4757 - val_accuracy: 0.8485
Epoch 19/20
248/248 [==============================] - 77s 310ms/step - loss: 0.2563 - accuracy: 0.9129 - val_loss: 0.4316 - val_accuracy: 0.8657
Epoch 20/20
248/248 [==============================] - 79s 317ms/step - loss: 0.2434 - accuracy: 0.9139 - val_loss: 0.4313 - val_accuracy: 0.8566
4/4 [==============================] - 4s 33ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


In [5]:
def run_experiment(model):



    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




    history = model.fit(
        train_generator,
        batch_size=64,
        epochs=15,
        validation_data=validation_generator,

    )
    

    model.trainable = True
    history = model.fit(
        train_generator,
        batch_size=64,
        epochs=16 // 2,

        validation_data=validation_generator,

    )

    accuracy = model.evaluate(test_generator)
    


    #model.save("./vit_best_model.h5")
    return history

vit_classifier = create_vit_classifier()
vit_classifier.summary()
history = run_experiment(vit_classifier)

c:\Users\badr0\.conda\envs\py310\lib\site-packages\vit_keras\vit.py:139: UserWarning: Can only use pretrained_top with imagenet21k+imagenet2012 if classes = 1000. Setting manually.
  warnings.warn(
c:\Users\badr0\.conda\envs\py310\lib\site-packages\vit_keras\utils.py:81: UserWarning: Resizing position embeddings from 12, 12 to 7, 7
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      7         
 al)                                                             
                                                                 
 vit-l32 (Functional)        (None, 1000)              306535400 
                                                                 
 dense (Dense)               (None, 512)               513024    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131584    
                                                             

c:\Users\badr0\.conda\envs\py310\lib\site-packages\vit_keras\vit.py:139: UserWarning: Can only use pretrained_top with imagenet21k+imagenet2012 if classes = 1000. Setting manually.
  warnings.warn(
c:\Users\badr0\.conda\envs\py310\lib\site-packages\vit_keras\utils.py:81: UserWarning: Resizing position embeddings from 12, 12 to 7, 7
  warnings.warn(
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      7         
 al)                                                             
                                                                 
 vit-l32 (Functional)        (None, 1000)              306535400 
                                                                 
 dense (Dense)               (None, 512)               513024    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               33024     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 5)                 645       
                                                                 
=================================================================
Total params: 307,213,684
Trainable params: 678,277
Non-trainable params: 306,535,407
_________________________________________________________________
Epoch 1/15
248/248 [==============================] - 106s 340ms/step - loss: 0.8211 - accuracy: 0.6891 - val_loss: 0.6033 - val_accuracy: 0.7869
Epoch 2/15
248/248 [==============================] - 79s 320ms/step - loss: 0.5832 - accuracy: 0.7886 - val_loss: 0.6070 - val_accuracy: 0.7793
Epoch 3/15
248/248 [==============================] - 78s 314ms/step - loss: 0.5243 - accuracy: 0.8111 - val_loss: 0.5907 - val_accuracy: 0.7955
Epoch 4/15
248/248 [==============================] - 77s 311ms/step - loss: 0.5011 - accuracy: 0.8237 - val_loss: 0.5002 - val_accuracy: 0.8202
Epoch 5/15
248/248 [==============================] - 78s 314ms/step - loss: 0.4595 - accuracy: 0.8380 - val_loss: 0.5220 - val_accuracy: 0.8131
Epoch 6/15
248/248 [==============================] - 77s 311ms/step - loss: 0.4439 - accuracy: 0.8399 - val_loss: 0.5035 - val_accuracy: 0.8146
Epoch 7/15
248/248 [==============================] - 78s 312ms/step - loss: 0.4468 - accuracy: 0.8361 - val_loss: 0.5615 - val_accuracy: 0.8000
Epoch 8/15
248/248 [==============================] - 77s 310ms/step - loss: 0.4288 - accuracy: 0.8475 - val_loss: 0.4769 - val_accuracy: 0.8359
Epoch 9/15
248/248 [==============================] - 78s 313ms/step - loss: 0.4056 - accuracy: 0.8558 - val_loss: 0.5211 - val_accuracy: 0.8237
Epoch 10/15
248/248 [==============================] - 77s 309ms/step - loss: 0.3949 - accuracy: 0.8578 - val_loss: 0.4823 - val_accuracy: 0.8232
Epoch 11/15
248/248 [==============================] - 78s 312ms/step - loss: 0.3767 - accuracy: 0.8677 - val_loss: 0.4776 - val_accuracy: 0.8323
Epoch 12/15
248/248 [==============================] - 77s 309ms/step - loss: 0.3841 - accuracy: 0.8634 - val_loss: 0.4976 - val_accuracy: 0.8323
Epoch 13/15
248/248 [==============================] - 76s 306ms/step - loss: 0.3803 - accuracy: 0.8679 - val_loss: 0.4740 - val_accuracy: 0.8434
Epoch 14/15
248/248 [==============================] - 76s 307ms/step - loss: 0.3621 - accuracy: 0.8689 - val_loss: 0.4826 - val_accuracy: 0.8303
Epoch 15/15
248/248 [==============================] - 78s 313ms/step - loss: 0.3706 - accuracy: 0.8679 - val_loss: 0.4398 - val_accuracy: 0.8475
Epoch 1/8
248/248 [==============================] - 78s 313ms/step - loss: 0.3580 - accuracy: 0.8691 - val_loss: 0.4367 - val_accuracy: 0.8465
Epoch 2/8
248/248 [==============================] - 77s 309ms/step - loss: 0.3637 - accuracy: 0.8665 - val_loss: 0.4756 - val_accuracy: 0.8379
Epoch 3/8
248/248 [==============================] - 77s 310ms/step - loss: 0.3535 - accuracy: 0.8753 - val_loss: 0.4543 - val_accuracy: 0.8374
Epoch 4/8
248/248 [==============================] - 78s 313ms/step - loss: 0.3446 - accuracy: 0.8761 - val_loss: 0.4766 - val_accuracy: 0.8348
Epoch 5/8
248/248 [==============================] - 77s 310ms/step - loss: 0.3340 - accuracy: 0.8819 - val_loss: 0.4694 - val_accuracy: 0.8359
Epoch 6/8
248/248 [==============================] - 78s 313ms/step - loss: 0.3528 - accuracy: 0.8801 - val_loss: 0.4150 - val_accuracy: 0.8596
Epoch 7/8
248/248 [==============================] - 77s 311ms/step - loss: 0.3229 - accuracy: 0.8851 - val_loss: 0.4646 - val_accuracy: 0.8444
Epoch 8/8
248/248 [==============================] - 78s 313ms/step - loss: 0.3425 - accuracy: 0.8787 - val_loss: 0.4841 - val_accuracy: 0.8394
4/4 [==============================] - 3s 36ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


In [6]:
def run_experiment(model):



    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




    history = model.fit(
        train_generator,
        batch_size=16,
        epochs=8,
        validation_data=validation_generator,

    )
    

    model.trainable = True
    history = model.fit(
        train_generator,
        batch_size=16,
        epochs=8 // 2,

        validation_data=validation_generator,

    )

    accuracy = model.evaluate(test_generator)
    


    #model.save("./vit_best_model.h5")
    return history

vit_classifier = create_vit_classifier()
vit_classifier.summary()
history = run_experiment(vit_classifier)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      7         
 al)                                                             
                                                                 
 vit-l32 (Functional)        (None, 1000)              306535400 
                                                                 
 dense_4 (Dense)             (None, 512)               513024    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131584    
                                                           

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      7         
 al)                                                             
                                                                 
 vit-l32 (Functional)        (None, 1000)              306535400 
                                                                 
 dense_4 (Dense)             (None, 512)               513024    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131584    
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               33024     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 5)                 645       
                                                                 
=================================================================
Total params: 307,213,684
Trainable params: 678,277
Non-trainable params: 306,535,407
_________________________________________________________________
Epoch 1/8
248/248 [==============================] - 100s 334ms/step - loss: 0.8291 - accuracy: 0.6845 - val_loss: 0.6064 - val_accuracy: 0.7727
Epoch 2/8
248/248 [==============================] - 448s 2s/step - loss: 0.5815 - accuracy: 0.7880 - val_loss: 0.5853 - val_accuracy: 0.7904
Epoch 3/8
248/248 [==============================] - 76s 306ms/step - loss: 0.5341 - accuracy: 0.8115 - val_loss: 0.5447 - val_accuracy: 0.8000
Epoch 4/8
248/248 [==============================] - 77s 309ms/step - loss: 0.5084 - accuracy: 0.8211 - val_loss: 0.5313 - val_accuracy: 0.8116
Epoch 5/8
248/248 [==============================] - 76s 305ms/step - loss: 0.4642 - accuracy: 0.8331 - val_loss: 0.5587 - val_accuracy: 0.7939
Epoch 6/8
248/248 [==============================] - 77s 310ms/step - loss: 0.4457 - accuracy: 0.8427 - val_loss: 0.4729 - val_accuracy: 0.8308
Epoch 7/8
248/248 [==============================] - 76s 304ms/step - loss: 0.4397 - accuracy: 0.8434 - val_loss: 0.4987 - val_accuracy: 0.8263
Epoch 8/8
248/248 [==============================] - 76s 308ms/step - loss: 0.4254 - accuracy: 0.8489 - val_loss: 0.4951 - val_accuracy: 0.8222
Epoch 1/4
248/248 [==============================] - 77s 309ms/step - loss: 0.4157 - accuracy: 0.8525 - val_loss: 0.4452 - val_accuracy: 0.8328
Epoch 2/4
248/248 [==============================] - 76s 307ms/step - loss: 0.4045 - accuracy: 0.8556 - val_loss: 0.4829 - val_accuracy: 0.8399
Epoch 3/4
248/248 [==============================] - 75s 304ms/step - loss: 0.3948 - accuracy: 0.8564 - val_loss: 0.5177 - val_accuracy: 0.8172
Epoch 4/4
248/248 [==============================] - 77s 309ms/step - loss: 0.3744 - accuracy: 0.8697 - val_loss: 0.4516 - val_accuracy: 0.8328
4/4 [==============================] - 3s 39ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
